# Mean values for separate channels

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import skimage.io
from PIL import Image

dataset = "bbbc022"
channels = ["DNA", "RNA", "ER", "AGP", "Mito"]

# Read metadata
meta = pd.read_csv(f"../{dataset}/inputs/metadata/index.csv")

In [2]:
# One-Time operation, ~ 20min
# For each channel read all images and save each image's mean, max and min
means = {"channel": channels, "images_mean": [], "images_max": [], "images_min": [], "images_max_mean": [], "images_min_mean": []}

for channel in channels:
    img_means = []
    img_max = []
    img_min = []

    for image in meta[channel]:
        img = skimage.io.imread(f"../{dataset}/inputs/images/{image}")
        img_means.append(img.mean())
        img_max.append(img.max())
        img_min.append(img.min())
    
    img_mean = np.array(img_means)
    img_max = np.array(img_max)
    img_min = np.array(img_min)
    
    means["images_mean"].append(img_mean.mean())
    means["images_max"].append(img_max.max())
    means["images_min"].append(img_min.min())
    means["images_max_mean"].append(img_max.mean())
    means["images_min_mean"].append(img_min.mean())

# Save means, max and min to csv
df = pd.DataFrame(means)
df.to_csv(f"channel_means_{dataset}.csv", index=False)

In [3]:
# Read mean, max and min from csv
df = pd.read_csv(f"channel_means_{dataset}.csv")

In [4]:
# One-Time operation
# Create new mean images for each channel

# Read a sample image to keep original metadata and shape
sample_image = Image.open(f"../{dataset}/inputs/images/20585/IXMtest_A01_s1_w164FBEEF7-F77C-4892-86F5-72D0160D4FB2.tif")

for channel in channels:
    channel_mean = int(df[df.channel == channel]['images_mean'].item())
    new_image = Image.new(sample_image.mode, sample_image.size, channel_mean)
    new_image.info.update(sample_image.info)
    new_image.save(f"../{dataset}/inputs/images/{channel}_mean.tif", **sample_image.info)

In [5]:
# One-Time operation
# Create index metadata files for DeepProfiler

for channel in channels:
    channel_meta = meta.copy()
    channel_meta[channel] = f"{channel}_mean.tif"
    channel_meta.to_csv(f"../{dataset}/inputs/metadata/index_{channel}_mean.csv", index=False)